In [42]:
import pandas as pd
import MeCab
import mojimoji
import re
import os
from collections import Counter

In [2]:
DATA_DIR = '../data/livedoor-homme'

In [3]:
df = pd.read_csv('../livedoornews.csv')

In [4]:
df.head()

,url,datetime,title,body,media
0,http://news.livedoor.com/article/detail/5978741/,2011-10-30T10:15:00+0900,【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か,2005年11月から翌2006年7月まで読売新聞にて連載された、直木賞作家・角田光代による初...,movie-enter
1,http://news.livedoor.com/article/detail/6322901/,2012-02-29T11:45:00+0900,藤原竜也、中学生とともにロケット打ち上げに成功,「アンテナを張りながら生活をしていけばいい」\n2月28日、映画『おかえり、はやぶさ』（3月...,movie-enter
2,http://news.livedoor.com/article/detail/6176324/,2012-01-09T14:00:00+0900,『戦火の馬』ロイヤル・プレミアにウィリアム王子＆キャサリン妃が出席,3月2日より全国ロードショーとなる、スティーブン・スピルバーグの待望の監督最新作『戦火の馬』...,movie-enter
3,http://news.livedoor.com/article/detail/6573929/,2012-05-19T12:00:00+0900,香里奈、女子高生100人のガチンコ質問に回答「ラーメンも食べる」,女優の香里奈が18日、都内で行われた映画『ガール』（5月26日公開）の女子高生限定試写会にサ...,movie-enter
4,http://news.livedoor.com/article/detail/5914880/,2011-10-05T19:11:00+0900,ユージの前に立ちはだかったJOY「僕はAKBの高橋みなみを守る」,5日、東京・千代田区の内幸町ホールにて、映画『キャプテン・アメリカ/ザ・ファースト・アベンジ...,movie-enter


In [51]:
#https://qiita.com/arata-honda/items/6409e387027c31365741
#colabとlocalは後で設定で使い分けられるようにする
class WordDividor:
    INDEX_CATEGORY = 0
    #TARGET_CATEGORY = ['名詞', '動詞', '形容詞']

    def __init__(self, dictionary="mecabrc"):
        self.dictionary = dictionary
        
        if dictionary == 'mecabrc':
            self.tagger = MeCab.Tagger()
        elif dictionary == 'neologd':
            self.tagger = MeCab.Tagger(f"-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")

    def extract_words_for_google_colab(self, text):
        if not text:
            return []

        # テキストの余分なデータの削除(ex : httpなど)
        text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
        text = re.sub(r'[!-~]', "", text)  # 半角記号,数字,英字
        text = re.sub(r'[︰-＠]', "", text)  # 全角記号
        text = re.sub('\n', " ", text)  # 改行文字

        self.tagger.parse("")
        node = self.tagger.parseToNode(text)
        words = []
        while node:
            if node.feature.split(",")[0] == u"名詞":
                #print(node.feature)
                word = node.feature.split(',')[6]
                #print(word)
                words.append(word)
            elif node.feature.split(",")[0] == u"形容詞":
                #print(node.feature)
                word = node.feature.split(',')[6]
                #print(word)
                words.append(word)
            elif node.feature.split(",")[0] == u"動詞":
                #print(node.feature)
                word = node.feature.split(',')[6]
                #print(word)
                words.append(word)
            node = node.next
        return words
    
    def extract_words_for_local(self, text):
        if not text:
            return []

        # テキストの余分なデータの削除(ex : httpなど)
        text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
        text = re.sub(r'[!-~]', "", text)  # 半角記号,数字,英字
        text = re.sub(r'[︰-＠]', "", text)  # 全角記号
        text = re.sub('\n', " ", text)  # 改行文字

        self.tagger.parse("")
        node = self.tagger.parseToNode(text)
        words = []
        while node:
            if node.feature.split(",")[0] == u"名詞":
                #print(node.feature)
                word = node.feature.split(',')[5]
                #print(word)
                words.append(word)
            elif node.feature.split(",")[0] == u"形容詞":
                #print(node.feature)
                word = node.feature.split(',')[5]
                #print(word)
                words.append(word)
            elif node.feature.split(",")[0] == u"動詞":
                #print(node.feature)
                word = node.feature.split(',')[5]
                #print(word)
                words.append(word)
            node = node.next
        return words


## 前処理

In [78]:
#全角を半角に統一
df['text'] = df['body'].apply(lambda text: mojimoji.zen_to_han(text))

In [79]:
#形態素解析
wd = WordDividor('neologd')
#df['text'] = df['text'].apply(lambda text: wd.extract_words_for_local(text))
df['text'] = df['text'].apply(lambda text: wd.extract_words_for_google_colab(text))

In [81]:
df['text'] = df['text'].apply(lambda text: ' '.join(text))

In [82]:
text_list = [text for text in df['text'].tolist()]
words = (' '.join(text_list)).split()

In [83]:
#先行研究の語彙数は34772(文章中に5回以上出現しない単語は削除した場合)
len(set(words))

45442

## 保存

In [84]:
df.to_csv('../output/preprocessed_data.csv', index=False)